In [1]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.utils import shuffle
import numpy as np
from datasets import *
# from models_SS import *
from models_MR import LapRVFL, LapDeepRVFL, LapEnsembleDeepRVFL, LapBRVFL, LapBDeepRVFL, LapBEnsembleDeepRVFL
from time import time
from run_SS_helpers import *


# set random seed
np.random.seed(42)

partition = [40, 1000, 40, 360]
lap_lams = [[2**i, 2**j] for i in range(-6, 13, 2) for j in range(-6, 13, 2)] # permutation of all possible lams
NN = 2
data, label, n_class = coil20()
run_LapRVFL(data, label, n_class, partition, NN)


running Laplacian RVFL...
Unlab:  0.8366666666666666 ± 0.020917828652982966
Val:  0.875 ± 0.05496210815947049
Test:  0.8358796296296296 ± 0.03455437827973525
Lambda:  [1024, 1]
Train time:  0.47211239735285443


In [2]:
run_LapdRVFL(data, label, n_class, partition, NN)

running Laplacian Deep RVFL...
Unlab:  0.7932499999999999 ± 0.04431539424022612
Val:  0.8291666666666666 ± 0.026679684322636874
Test:  0.8002314814814816 ± 0.028064683455831005
Lambda:  [4096, 0.0625]
Train time:  3.6970196664333344


In [3]:
run_LapedRVFL(data, label, n_class, partition, NN)

running Laplacian Ensemble Deep RVFL...
Unlab:  0.8582499999999998 ± 0.02367884076554425
Val:  0.8604166666666667 ± 0.051496696226284486
Test:  0.8312499999999999 ± 0.02904792850111281
Lambda:  [16, 1]
Train time:  6.719910204410553


In [1]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.utils import shuffle
import numpy as np
from datasets import *
from models_SS import *
from models_MR import LapRVFL, LapDeepRVFL, LapEnsembleDeepRVFL, LapBRVFL, LapBDeepRVFL, LapBEnsembleDeepRVFL
from time import time
from run_SS_helpers import *

data, label, n_class = coil20()
np.random.seed(42)

In [3]:
partition = [40, 1000, 40, 360]
lam = 1
n_layer = 1
activation='sigmoid'
model_class = DeepRVFL
lams = [2**i for i in range(-12, 13, 2)] # regularization parameter, lambda

unlab = []
val = []
test = []
t = []
for i, lam in enumerate(lams):
    unlab_acc, val_acc, test_acc, train_time = run_SS(data, label, n_class, model_class, partition, lam=lam, n_layer=5, activation='sigmoid')
    unlab.append(unlab_acc)
    val.append(val_acc)
    test.append(test_acc)
    t.append(train_time)

max_index = np.argmax(val, axis=0)[0]
opt_lam = lams[max_index]
print('Unlab: ', unlab[max_index][0], u"\u00B1", unlab[max_index][1])
print('Val: ', val[max_index][0], u"\u00B1", val[max_index][1])
print('Test: ', test[max_index][0], u"\u00B1", test[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index])


Unlab:  0.6594166666666667 ± 0.023386101047892155
Val:  0.7041666666666666 ± 0.06909634979907084
Test:  0.6349537037037036 ± 0.04842501858083466
Lambda:  16
Train time:  1.387277513742447


In [5]:
# RVFL

n_node = 10 # num of nodes in hidden layer
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = RVFL(n_node, lam, w_range, b_range)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, True)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = RVFL(n_node, lam, w_range, b_range)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, True)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9396551724137931
Validation: 2
Validation accuracy: 0.9310344827586207
Validation: 3
Validation accuracy: 0.9478260869565217
Validation: 4
Validation accuracy: 0.9565217391304348
Validation: 5
Validation accuracy: 0.9391304347826087
Validation: 6
Validation accuracy: 0.9217391304347826
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9478260869565217
Validation: 9
Validation accuracy: 0.9478260869565217
Validation: 10
Validation accuracy: 0.9565217391304348

Test accuracy: 0.9236111111111112


In [6]:
# dRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 2 # num of hidden layers
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = DeepRVFL(n_node, lam, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, False)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = DeepRVFL(n_node, lam, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9396551724137931
Validation: 2
Validation accuracy: 0.9482758620689655
Validation: 3
Validation accuracy: 0.9391304347826087
Validation: 4
Validation accuracy: 0.9652173913043478
Validation: 5
Validation accuracy: 0.9304347826086956
Validation: 6
Validation accuracy: 0.9217391304347826
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9304347826086956
Validation: 9
Validation accuracy: 0.9652173913043478
Validation: 10
Validation accuracy: 0.9652173913043478

Test accuracy: 0.9305555555555556


In [7]:
# edRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 2 # num of hidden layers
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = EnsembleDeepRVFL(n_node, lam, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, False)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = EnsembleDeepRVFL(n_node, lam, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9310344827586207
Validation: 2
Validation accuracy: 0.9482758620689655
Validation: 3
Validation accuracy: 0.9478260869565217
Validation: 4
Validation accuracy: 0.9565217391304348
Validation: 5
Validation accuracy: 0.9217391304347826
Validation: 6
Validation accuracy: 0.9304347826086956
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9565217391304348
Validation: 9
Validation accuracy: 0.9652173913043478
Validation: 10
Validation accuracy: 0.9391304347826087

Test accuracy: 0.9236111111111112


In [3]:
# BRVFL

n_node = 10 # num of nodes in hidden layer
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    brvfl = BRVFL(n_node, w_range, b_range)
    brvfl.train(X_val_train, y_val_train, n_class)
    prediction = brvfl.predict(X_val_test, True)
    acc = brvfl.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
brvfl = BRVFL(n_node, w_range, b_range)
brvfl.train(X_train, y_train, n_class)
prediction = brvfl.predict(X_test, True)
acc = brvfl.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1



Convergence after  17  iterations
Validation accuracy: 0.896551724137931
Validation: 2



Convergence after  16  iterations
Validation accuracy: 0.8706896551724138
Validation: 3



Convergence after  17  iterations
Validation accuracy: 0.9217391304347826
Validation: 4



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 5


...


Convergence after  17  iterations
Validation accuracy: 0.9304347826086956
Validation: 8



Convergence after  17  iterations
Validation accuracy: 0.8782608695652174
Validation: 9



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 10



Convergence after  17  iterations
Validation accuracy: 0.8869565217391304



Convergence after  14  iterations

Test accuracy: 0.9166666666666666


In [2]:
# BdRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 5 # num of hidden layers
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = BayesianDeepRVFL(n_node, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, True)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = BayesianDeepRVFL(n_node, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, True)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1



Convergence after  16  iterations
Validation accuracy: 0.9051724137931034
Validation: 2



Convergence after  16  iterations
Validation accuracy: 0.8793103448275862
Validation: 3



Convergence after  15  iterations
Validation accuracy: 0.9391304347826087
Validation: 4



Convergence after  16  iterations
Validation accuracy: 0.9565217391304348
Validation: 5


...


Convergence after  16  iterations
Validation accuracy: 0.9043478260869565
Validation: 8



Convergence after  16  iterations
Validation accuracy: 0.9130434782608695
Validation: 9



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 10



Convergence after  16  iterations
Validation accuracy: 0.9043478260869565



Convergence after  16  iterations

Test accuracy: 0.8645833333333334


In [2]:
# BedRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 5 # num of hidden layers
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

# for i, kf_values in enumerate(kf.split(X_train, y_train)):
# #     print(f'train: {train_index}, val: {val_index}')
#     print('Validation: {}'.format(i + 1))
#     train_index, val_index = kf_values
#     X_val_train, X_val_test = X_train[train_index], X_train[val_index]
#     y_val_train, y_val_test = y_train[train_index], y_train[val_index]
#     model = BayesianEnsembleDeepRVFL(n_node, w_range, b_range, n_layer)
#     model.train(X_val_train, y_val_train, n_class)
#     prediction = model.predict(X_val_test, False)
#     acc = model.eval(X_val_test, y_val_test)
#     print(f'Validation accuracy: {acc}')
#     val_acc.append(acc)
#     if acc >= max(val_acc):
#         max_index = train_index

# X_train, y_train = X_train[max_index], y_train[max_index]
model = BayesianEnsembleDeepRVFL(n_node, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')


Convergence after  17  iterations



Convergence after  16  iterations



Convergence after  17  iterations



Convergence after  17  iterations



Convergence after  17  iterations

Test accuracy: 0.9201388888888888
